### Testing the MOJO file 

## Install packages 

!pip install  https://s3.amazonaws.com/h2o-release/datatable/stable/datatable-0.8.0/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl



!pip install /home/jorge/H2OExamples/mojos/daimojo-2.1.4+master.10-cp36-cp36m-linux_x86_64.whl

In [1]:
# import the daimojo model package
import daimojo.model
# import the datatable module
import datatable as dt
import os

#### Load the License to the enviroment

In [2]:
with open('/home/jorge/H2OExamples/license.sig', 'r') as f:
    dai_license = f.read().replace('\n','') 
    
os.environ["DRIVERLESS_AI_LICENSE_KEY"] = dai_license

#### Load the MOJO and make predictions

In [3]:
# specify the location of the MOJO
model = daimojo.model("/home/jorge/H2OExamples/mojos/mojo-pipeline/pipeline.mojo")

# Check Mojo Info
print(model.created_time)
print(model.feature_names)


Mon Sep  9 14:19:27 2019
['LIMIT_BAL', 'EDUCATION', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [4]:
# parse the example.csv file to test
pydt = dt.fread("/home/jorge/H2OExamples/mojos/mojo-pipeline/example.csv", na_strings=model.missing_values)
pydt

,LIMIT_BAL,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,…,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,40000,0,24,5,6,3,−2,3,6,−300,…,3,57,34,28,100
1,50000,1,21,8,6,2,4,4,0,−1860,…,14,54,5,45,22
2,30000,5,27,8,6,5,3,4,7,−1540,…,2,58,97,10,12
3,60000,0,23,8,5,0,4,2,0,−200,…,156,11,162,10,22
4,80000,0,30,8,4,−2,3,−2,7,−1540,…,100,54,6,31,104
5,60000,0,28,0,3,2,3,−1,−2,−2000,…,1,11,191,4,152
6,10000,0,28,4,−1,−1,3,2,7,−300,…,1,57,0,4,152
7,100000,0,29,2,0,7,2,2,4,−300,…,0,4,6,4,0
8,80000,0,29,3,6,1,−2,3,4,−1100,…,0,2,97,0,9


In [5]:
# make predictions on the example.csv file
res = model.predict(pydt)

# retrieve the predictions
res

,default payment next month.0,default payment next month.1
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0.644105,0.355895
1,0.660961,0.339039
2,0.722779,0.277221
3,0.643502,0.356498
4,0.720359,0.279641
5,0.643305,0.356695
6,0.698448,0.301552
7,0.627489,0.372511
8,0.601439,0.398561


In [6]:
res.to_list()  #format to return

[[0.6441048607230186,
  0.6609608605504036,
  0.72277856990695,
  0.6435021981596947,
  0.72035913169384,
  0.6433048248291016,
  0.6984476372599602,
  0.6274885684251785,
  0.6014387384057045,
  0.713371466845274],
 [0.35589513927698135,
  0.3390391394495964,
  0.27722143009305,
  0.35649780184030533,
  0.27964086830616,
  0.35669517517089844,
  0.3015523627400398,
  0.37251143157482147,
  0.3985612615942955,
  0.28662853315472603]]

### Deploy to Azure 



In [7]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.62


In [8]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep='\n')

jorge_aml_workspace
my_aml_rg
eastus2
92429150-401a-431f-8955-e69c0c119e68



### Register Model

You can add tags and descriptions to your models.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.


In [9]:
from azureml.core.model import Model

model = Model.register(model_path="./scoring-pipeline/pipeline.mojo",
                       model_name="MyMojoModel",
                       tags={'area': "financial", 'type': "classification"},
                       description="Credit Card Demo",
                       workspace=ws)

Registering model MyMojoSept9


Prepare the files to submit the docker instance: 

    * DAI License
    * Mojo Pipeline
    * datatable and c++ runtime dependencies    
    * Score.py
    * Conda Enviroment

In [10]:
import os

source_directory = "./scoring-pipeline/"

os.makedirs(source_directory, exist_ok=True)
os.makedirs("./scoring-pipeline/env", exist_ok=True)

In [11]:
!cp license.sig ./scoring-pipeline/env/
!cp mojos/mojo-pipeline/pipeline.mojo ./scoring-pipeline/
!cp mojos/daimojo-2.1.4+master.10-cp36-cp36m-linux_x86_64.whl ./scoring-pipeline/env/
!cp mojos/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl ./scoring-pipeline/env/

#### Score.py 

The score.py is the entry point to our deployment

In [1]:
%%writefile ./scoring-pipeline/score.py

import os
import json
from azureml.core.model import Model
import pandas as pd

def init():
    
    with open('./scoring-pipeline/env/license.sig', 'r') as f:
        dai_license = f.read().replace('\n','') 
    
    os.environ["DRIVERLESS_AI_LICENSE_KEY"] = dai_license    
    
    import daimojo.model

    global model
    model = daimojo.model("./scoring-pipeline/pipeline.mojo")
    

def run(raw_data):
    try:
        import datatable as dt
        data = json.loads(raw_data)['data']
        data = pd.read_json(data, orient='records')        
        pydt = dt.Frame(data)
        
        res = model.predict(pydt)
        
        return res.to_list()

    except Exception as e:
        error = str(e)
        return error

Overwriting ./scoring-pipeline/score.py


### Create Enviroment 


We are going to leverage Azure ML Services to create a new conda enviroment with all the dependencies needed including datatable and the mojo cpp package  

In [16]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="daienv")
conda_dep = CondaDependencies()

conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")

whl_url = Environment.add_private_pip_wheel(workspace=ws,file_path = "./scoring-pipeline/env/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl", exist_ok=True)
conda_dep.add_pip_package(whl_url)

whl_url = Environment.add_private_pip_wheel(workspace=ws,file_path = "./scoring-pipeline/env/daimojo-2.1.4+master.10-cp36-cp36m-linux_x86_64.whl", exist_ok=True)
conda_dep.add_pip_package(whl_url)


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

In [17]:
myenv.register(workspace=ws)

{
    "name": "daienv",
    "version": "5",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults",
                        "https://jorgeamlstoragebmumglkv.blob.core.windows.net/azureml/Environment/azureml-private-packages/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl",
                        "https://jorgeamlstoragebmumglkv.blob.core.windows.net/azureml/Environment/azureml-private-packages/daimojo-2.1.4+master.10-cp36-cp36m-linux_x86_64.whl"
                    ]
                },
                "numpy",
                "pandas"
           

In [18]:
from azureml.core import Image
build = myenv.build(ws)
build.wait_for_completion(show_output=True)

Image Build Status: Running

2019/09/30 20:01:45 Downloading source code...
2019/09/30 20:01:46 Finished downloading source code
2019/09/30 20:01:47 Using acb_vol_8690097a-e164-407a-b026-e344e286f432 as the home volume
2019/09/30 20:01:47 Creating Docker network: acb_default_network, driver: 'bridge'
2019/09/30 20:01:47 Successfully set up Docker network: acb_default_network
2019/09/30 20:01:47 Setting up Docker configuration...
2019/09/30 20:01:48 Successfully set up Docker configuration
2019/09/30 20:01:48 Logging in to registry: jorgeamlacrwgvevbct.azurecr.io
2019/09/30 20:01:50 Successfully logged into jorgeamlacrwgvevbct.azurecr.io
2019/09/30 20:01:50 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2019/09/30 20:01:50 Scanning for dependencies...
2019/09/30 20:01:50 Successfully scanned dependencies
2019/09/30 20:01:50 Launching container with name: acb_step_0
Sending build context to Docker daemon   59.9kB

Step 1/15 : FROM

### Create Inference Configuration

    source_directory = holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
    runtime = Which runtime to use for the image. Current supported runtimes are 'spark-py' and 'python
    entry_script = contains logic specific to initializing your model and running predictions
    envoriment = Our previously created enviroment to run our script



In [19]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory="./scoring-pipeline",
                                   environment=myenv, 
                                   entry_script="score.py")

## Deployment Options 


Currently there are 3 services available on AzureML. 

    * LocalWebService: ideal for testing.
    * Azure Container Instances: ideal for development and testing. 
    * Azure Kubernetes Service: ideal for dev/test and production 

### Deploy to Azure Container Instances (ACI) 

In [20]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Running............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [21]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2019-09-30T20:08:33,985351488+00:00 - iot-server/run 
2019-09-30T20:08:33,988264394+00:00 - rsyslog/run 
2019-09-30T20:08:33,994995110+00:00 - nginx/run 
/bin/bash: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libtinfo.so.5: no version information available (required by /bin/bash)
bash: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libtinfo.so.5: no version information available (required by bash)
/usr/sbin/nginx: /azureml-envs/azureml_6d0db39ff90a11d335175a34edbc28c4/lib/libcrypto.so.1.0.0: no version information available (required by /usr

Test ACI web service

In [22]:
print('web service hosted in ACI:', service.scoring_uri)

web service hosted in ACI: http://b90beb03-4d5f-46ce-9561-afdbf6a35879.eastus2.azurecontainer.io/score


In [23]:
import json
import pandas as pd

test_file = pd.read_csv("/home/jorge/H2OExamples/mojos/mojo-pipeline/example.csv",index_col=None)

test_sample = json.dumps({"data": test_file.to_json(orient='records')})



In [24]:
prediction = service.run(input_data = test_sample)
print(prediction)

[[0.6441048607230186, 0.6609608605504036, 0.72277856990695, 0.6435021981596947, 0.72035913169384, 0.6433048248291016, 0.6984476372599602, 0.6274885684251785, 0.6014387384057045, 0.713371466845274], [0.35589513927698135, 0.3390391394495964, 0.27722143009305, 0.35649780184030533, 0.27964086830616, 0.35669517517089844, 0.3015523627400398, 0.37251143157482147, 0.3985612615942955, 0.28662853315472603]]


### Deploy to Azure Kubernetes Service (AKS) 

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'myaks'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model

aks_target = AksCompute(ws,"myaks")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

### Deploy Model as a Local Docker Web Service

Make sure you have Docker installed and running.

Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)



In [ ]:
local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [ ]:
print('Local service port: {}'.format(local_service.port))

### Check Status and Get Container Logs

In [ ]:
print(local_service.get_logs())

In [ ]:
local_service.delete()